# Scratch code for testing purposes and small experiments. 


In [52]:


import numpy as np
import importlib
from helpers.Graph import Graph
import pandas as pd
import random
from tqdm import tqdm

%load_ext autoreload
%autoreload 2


def common_neighbors(adj):
    return np.dot(adj, adj)

def eigenvalues(adj):
    return np.linalg.eigvals(adj)

def eigenvectors(adj):
    return np.linalg.eig(adj)




m1 = np.array([[0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
                    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 1, 0, 1, 0, 0, 1],
                    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
                    [0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                    [1, 0, 0, 1, 0, 0, 0, 0, 1, 1],
                    [0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
                    [1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
                    [0, 0, 1, 0, 1, 1, 0, 0, 1, 0]])


m2 = np.array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
                    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
                    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
                    [0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                    [0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
                    [1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
                    [1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
                    [0, 0, 1, 0, 1, 0, 1, 0, 1, 0]])





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Eigen values of G = G1 U G2 equal to the union of eigen values of G1 and G2 ?

In [53]:
eig_g1 = eigenvalues(m1)
eig_g2 = eigenvalues(m2)

m = np.logical_or(m1, m2).astype(int)

print("Eigen values of G1", eig_g1)
print("Eigen values of G2", eig_g2)
print("Eigen values of G", np.linalg.eigvals(m))



Eigen values of G1 [ 4.         -2.56155281 -2.23606798  2.23606798  1.56155281 -1.
 -1.          1.         -1.         -1.        ]
Eigen values of G2 [ 4.         -2.56155281 -2.23606798 -1.          2.23606798  1.
  1.56155281 -1.         -1.         -1.        ]
Eigen values of G [ 4.90768271  2.17008649  1.56914969 -2.83573973  0.31110782 -1.4811943
 -0.64109268 -2.         -1.         -1.        ]


Common neighbors matrix

In [54]:
A = common_neighbors(m) # or np.dot(m, m) for m^2
print("A = ", A)

A =  [[5 1 3 3 2 2 1 3 3 2]
 [1 4 1 1 1 3 3 2 0 2]
 [3 1 5 2 3 1 2 2 3 3]
 [3 1 2 4 1 2 2 2 2 1]
 [2 1 3 1 4 2 2 1 2 2]
 [2 3 1 2 2 6 5 3 2 2]
 [1 3 2 2 2 5 6 2 2 3]
 [3 2 2 2 1 3 2 5 2 3]
 [3 0 3 2 2 2 2 2 4 2]
 [2 2 3 1 2 2 3 3 2 5]]


In [55]:
m1 = np.array([     [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0 , 1], #0
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1 , 0], #1
                    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0 , 0], #2
                    [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0 , 0], #3
                    [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0 , 0], #4
                    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0 , 0], #5
                    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0 , 0], #6
                    [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0 , 0], #7
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0 , 1], #8
                    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0 , 0], #9
                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0 , 1], #10
                    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1 , 0]]) #11


permutation = [0, 2, 4, 6, 8, 10, 1, 3, 5, 7, 9, 11]
m1_perm = m1[permutation, :][:, permutation]
print("m1**2 = ", np.dot(m1, m1))
print("m1_perm**2 = ", np.dot(m1_perm, m1_perm))

m1**2 =  [[3 0 2 0 1 0 0 0 1 0 2 0]
 [0 3 0 2 0 1 0 0 0 1 0 2]
 [2 0 3 0 2 0 1 0 0 0 1 0]
 [0 2 0 3 0 2 0 1 0 0 0 1]
 [1 0 2 0 3 0 2 0 1 0 0 0]
 [0 1 0 2 0 3 0 2 0 1 0 0]
 [0 0 1 0 2 0 3 0 2 0 0 0]
 [0 0 0 1 0 2 0 3 0 2 0 1]
 [1 0 0 0 1 0 2 0 3 0 1 0]
 [0 0 0 0 0 1 0 2 0 2 0 1]
 [2 0 1 0 0 0 1 0 2 0 2 0]
 [0 2 0 1 0 0 0 1 0 2 0 3]]
m1_perm**2 =  [[3 2 1 0 1 2 0 0 0 0 0 0]
 [2 3 2 1 0 1 0 0 0 0 0 0]
 [1 2 3 2 1 0 0 0 0 0 0 0]
 [0 1 2 3 2 0 0 0 0 0 0 0]
 [1 0 1 2 3 1 0 0 0 0 0 0]
 [2 1 0 1 2 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 2 1 0 1 2]
 [0 0 0 0 0 0 2 3 2 1 0 1]
 [0 0 0 0 0 0 1 2 3 2 1 0]
 [0 0 0 0 0 0 0 1 2 3 2 1]
 [0 0 0 0 0 0 0 0 1 2 2 1]
 [0 0 0 0 0 0 2 1 0 1 2 3]]


## Block matrix generation

In [56]:

block = Graph.block_from_txt("datasets/block.txt")

p_flip = 0.1

for i in range(0, block.first_partition_size):
    for j in range(block.first_partition_size+1, block.adj_matrix.shape[1]):
        if block.adj_matrix[i, j] == 1 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

        elif block.adj_matrix[i, j] == 0 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

block.to_txt(f"datasets/block_{p_flip}.txt")




In [57]:

for i in range(6):
    block = generate_block_matrix(100, 10, i / 100)
    graph = Graph.from_adj_matrix(block)
    print("p_fip = ", i/100,"clustering coeff. = ", graph.global_clustering_coefficient())

NameError: name 'generate_block_matrix' is not defined

## Idea of detecting and using triangles in the graph

In [ ]:
def is_triangle_common_neighbors(common_neighbors, i, j, k):
    return common_neighbors[i, j] == 1 and common_neighbors[j, k] == 1 and common_neighbors[i, k] == 1 and common_neighbors[i, i] == 2 and common_neighbors[j, j] == 2 and common_neighbors[k, k] == 2

def is_triangle_adj(adj, i, j, k):
    return adj[i, j] and adj[j, k] and adj[i, k]

polblogs = Graph.from_txt("datasets/polblogs.txt")

common_neighbors_polblogs = common_neighbors(polblogs.adj_matrix)
adj_polblogs = polblogs.adj_matrix

num_triangles_common_neighbors = 0
num_triangles_adj = 0

triplets = []
triangles = []

for i in range(polblogs.adj_matrix.shape[0]):
    for j in range(i + 1, polblogs.adj_matrix.shape[0]):
        for k in range(j + 1, polblogs.adj_matrix.shape[0]):
            if is_triangle_common_neighbors(common_neighbors_polblogs, i, j, k) and ( adj_polblogs[i, j] or adj_polblogs[j, k] or adj_polblogs[i, k]):
                triplets.append((i, j, k))
                num_triangles_common_neighbors += 1
                triangles.append((i, j, k))

            if is_triangle_adj(adj_polblogs, i, j, k):
                num_triangles_adj += 1



In [131]:
polblogs = Graph.from_txt("datasets/flickr.txt")

degrees = np.sum(polblogs.adj_matrix, axis=0)
number_degree_1 = np.sum(degrees == 1)

print("Number of nodes with degree 1 = ", number_degree_1)

Number of nodes with degree 1 =  2


In [220]:
A = np.array([[0, 1, 1, 0, 1],
              [1, 0, 1, 1, 0],
              [1, 1, 0, 1, 1],
              [0, 1, 1, 0, 1],
              [1, 0, 1, 1, 0]])
              


A_square = np.dot(A, A)
A_cube = np.dot(A_square, A)

print("A^2 = \n ", A_square)
print("A^3 = \n ", A_cube)

A^2 = 
  [[3 1 2 3 1]
 [1 3 2 1 3]
 [2 2 4 2 2]
 [3 1 2 3 1]
 [1 3 2 1 3]]
A^3 = 
  [[4 8 8 4 8]
 [8 4 8 8 4]
 [8 8 8 8 8]
 [4 8 8 4 8]
 [8 4 8 8 4]]


## Exploring the spectral approach

In [216]:
dataset = "polblogs"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.1)
A = np.dot(G.adj_matrix, G.adj_matrix.T)
# G1_singular_values = np.linalg.svd(G1.adj_matrix, compute_uv=False)
U_A, S_A, V_A = np.linalg.svd(A, compute_uv=True)
# U_G, S_G, V_G = np.linalg.svd(G.adj_matrix, compute_uv=True)
U_G1, S_G1, V_G1 = np.linalg.svd(G1.adj_matrix, compute_uv=True)


## Sign assignment

In [101]:
M_star = np.zeros_like(G.adj_matrix)

for i in tqdm(range(S_A.shape[0])):
    # print("Singular value A = ", S_A[i])
    S_Gi_plus = np.sqrt(S_A[i])
    S_Gi_minus =  - np.sqrt(S_A[i])

    M_star_plus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_plus * V_A[i, :].reshape(1, -1)
    M_star_minus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_minus * V_A[i, :].reshape(1, -1)

    binary_M_star_plus = np.where(M_star_plus > 0.5, 1, 0)
    binary_M_star_minus = np.where(M_star_minus > 0.5, 1, 0)

    distance_plus = np.linalg.norm(M_star_plus - binary_M_star_plus, ord='fro')
    distance_minus = np.linalg.norm(M_star_minus - binary_M_star_minus, ord='fro')

    if distance_plus < distance_minus:
        M_star = M_star_plus
    else:
        M_star = M_star_minus

M_star = np.where(M_star > 0.5, 1, 0)
# print("M_star = ", M_star)

print("Number of errors : ", np.linalg.norm(G.adj_matrix - M_star) ** 2)

100%|██████████| 2485/2485 [02:31<00:00, 16.45it/s]

Number of errors :  69.0


## Revisited sign assignment

In [ ]:
# M_star = np.zeros_like(G.adj_matrix)
M_star = G1.adj_matrix

for i in tqdm(range(S_A.shape[0])):

    S_Gi_plus = np.sqrt(S_A[i])
    S_Gi_minus =  - np.sqrt(S_A[i])

    M_star_plus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_plus * V_A[i, :].reshape(1, -1)
    M_star_minus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_minus * V_A[i, :].reshape(1, -1)


    binary_M_star_plus = np.where(M_star_plus > 0.5, 1, 0)
    binary_M_star_minus = np.where(M_star_minus > 0.5, 1, 0)


    A_plus = np.dot(binary_M_star_plus, binary_M_star_plus.T)
    A_minus = np.dot(binary_M_star_minus, binary_M_star_minus.T)
    
    distance_plus = np.linalg.norm(A_plus - A, ord='fro')
    distance_minus = np.linalg.norm(A_minus - A, ord='fro')

    # print("----------------------")
    # print("A_plus = ", distance_plus)
    # print("A_minus = ", distance_minus)

    if distance_plus < distance_minus:
        M_star = M_star_plus
    else:
        M_star = M_star_minus

print(M_star)



In [217]:
def degree_attack(reconstructed_graph, A, degrees):
    number_modifs = 0
    for i in tqdm(range(A.shape[0]), desc="Degree attack"):
        if A[i, i] not in degrees:
            continue
        degree = np.sum(A[i]) 
        candidate = None
        possibilities = np.where(np.diag(A) <= degree - A[i, i] + 1)[0]
        for comb in combinations(possibilities, A[i, i]):
            sum_of_degrees = 0

            for k in comb:
                sum_of_degrees += A[k, k]

            if sum_of_degrees == degree:
                if candidate == None:
                    candidate = comb
                else:
                    candidate = None
                    break
        
        if candidate != None:
            for j in candidate:
                if reconstructed_graph.does_not_know_edge((i, j)):
                    reconstructed_graph.add_edge((i, j))
                    number_modifs += 1

    return reconstructed_graph, number_modifs

reconstructed_graph = graph1_copy(G1)

rec, modifs = degree_attack(reconstructed_graph, A, [1, 2])
print("Number of modifs = ", modifs)
print(A)

+-------------------------------+--------+
|              Stat             | Value  |
+-------------------------------+--------+
|   Number of impossible edges  |  1222  |
| Number of reconstructed edges |   0    |
|    Number of unknown edges    | 746031 |
+-------------------------------+--------+ 



Degree attack: 100%|██████████| 1222/1222 [00:03<00:00, 332.40it/s]

Number of modifs =  47
[[26  9  0 ...  0  0  0]
 [ 9 45  0 ...  0  0  0]
 [ 0  0  4 ...  0  1  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  1 ...  0 18  0]
 [ 0  0  0 ...  0  0  1]]


In [224]:
print(np.trace(G1.adj_matrix))

eigenvalues = np.linalg.eigvalsh(G.adj_matrix)

print(np.sum(eigenvalues))

0


/opt/homebrew/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: could not broadcast input array from shape (1222,1222) into shape (1222,)